# Introduction

This notebook introduces the main building blocks of the ``webgpu`` package
and walks from a minimal "draw a triangle" example to a custom renderer
that sends data from Python to the GPU.

You will see how to:

- create a WebGPU canvas in Jupyter with ``webgpu.jupyter.Draw``
- use existing renderer classes (for example ``TriangulationRenderer``) to get
  something on screen quickly,
- define your own ``Renderer`` subclass with WGSL shader code,
- connect Python data (NumPy arrays) to WGSL via buffer bindings,
- add simple GUI controls via ``scene.gui`` to interact with your scene.


## What is WebGPU

WebGPU is a new graphics API that provides high-performance access to the GPU
for rendering and computation. It is a low-level API in the same spirit as
Vulkan, Metal, and Direct3D 12, but is designed to be cross-platform and run
in the browser.

The Python project ``webgpu`` provides a Python wrapper and small rendering
framework on top of WebGPU, so that you can write interactive graphics and
compute code directly in Python and run it in a browser.

**Useful links**

- Check WebGPU support: https://webgpureport.org/
- WGSL Tour: https://google.github.io/tour-of-wgsl/
- Specification: https://www.w3.org/TR/webgpu/


## First example

Webgpu provides renderer classes for common geometrical objects. The
``Draw`` function creates a canvas and renders all given render objects
together into its context.

In the following cell we use a built-in ``TriangulationRenderer`` that
creates a triangle from three 3D points. Try changing the coordinates or
the color tuple and re-running the cell to see how the output updates.


In [ ]:
import webgpu.jupyter as wj
from webgpu.triangles import TriangulationRenderer

points = [(0, 0, 0), (0, 1, 0), (1, 0, 0)]
trigs = TriangulationRenderer(points, color=(1, 0, 0, 1))
wj.Draw(trigs)

## Hello World example

To implement a custom ``RenderObject`` one needs to implement
``get_shader_code`` and ``get_bounding_box``.
The shader code is written in WebGPU Shading Language
(https://www.w3.org/TR/WGSL/).

Data structure which is the output of the vertex shader and the input of the
fragment shader. The values are interpolated for each pixel of the triangle:

.. code-block:: rust

   struct FragmentInput {
       @builtin(position) p: vec4<f32>,
       @location(0) color: vec4<f32>,
   };

Vertex shader, returns a ``FragmentInput`` object (now with hard-coded vertex
positions and colors):

.. code-block:: rust

   @vertex
   fn vertex_main(
     @builtin(vertex_index) vertex_index : u32
) -> FragmentInput {

     var pos = array<vec4f, 3>(
       vec4f( 0.0,  0.5, 0., 1.),
       vec4f(-0.5, -0.5, 0., 1.),
       vec4f( 0.5, -0.5, 0., 1.)
     );

     var color = array<vec4f, 3>(
       vec4f(1., 0., 0., 1.),
       vec4f(0., 1., 0., 1.),
       vec4f(0., 0., 1., 1.)
     );

     return FragmentInput( pos[vertex_index], color[vertex_index] );
   }

Fragment shader function, called for each pixel of the final image; it just
returns the color of the input:

.. code-block:: rust

   @fragment
   fn fragment_main(input: FragmentInput) -> @location(0) vec4f {
     return input.color;
   }


In [ ]:
import webgpu.jupyter as wj
from webgpu.renderer import Renderer

shader_code = """
// Data structure which is the output of the vertex shader and the input of the fragment shader
struct FragmentInput {
    @builtin(position) p: vec4<f32>,
    @location(0) color: vec4<f32>,
};

// Vertex shader, returns a FragmentInput object
@vertex
fn vertex_main(
  @builtin(vertex_index) vertex_index : u32
) -> FragmentInput {

  var pos = array<vec4f, 3>(
    vec4f( 0.0,  0.5, 0., 1.),
    vec4f(-0.5, -0.5, 0., 1.),
    vec4f( 0.5, -0.5, 0., 1.)
  );

  var color = array<vec4f, 3>(
    vec4f(1., 0., 0., 1.),
    vec4f(0., 1., 0., 1.),
    vec4f(0., 0., 1., 1.)
  );
  
  return FragmentInput( pos[vertex_index], color[vertex_index] );
}

@fragment
fn fragment_main(input: FragmentInput) -> @location(0) vec4f {
  return input.color;
}
"""


class Triangle(Renderer):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.n_vertices = 3

    def get_bounding_box(self):
        return ((0, 0, 0), (1, 1, 1))

    def get_shader_code(self):
        return shader_code


wj.Draw(Triangle())

You can draw multiple objects into one scene:

In [ ]:
from webgpu.triangles import TriangulationRenderer

points = [(0, 0, 0), (0, 1, 0), (1, 0, 0)]
trigs = TriangulationRenderer(points, color=(1, 0, 0, 1))
wj.Draw([Triangle(), trigs])

## Camera

As you can see above, the hello-world triangle does not rotate with camera
movement. For this the class has to be extended to use the camera
functionality:

**Changes in shader code**

Import camera utility functions:

.. code-block:: rust

   #import camera

Apply ``cameraMapPoint`` to vertex position in the vertex shader:

.. code-block:: rust

   return FragmentInput( cameraMapPoint(pos[vertex_index]),
                         color[vertex_index] );


In [ ]:
import webgpu.jupyter as wj
from webgpu.renderer import Renderer

shader_code = """
#import camera

// Data structure which is the output of the vertex shader and the input of the fragment shader
struct FragmentInput {
    @builtin(position) p: vec4<f32>,
    @location(0) color: vec4<f32>,
};

// Vertex shader, returns a FragmentInput object
@vertex
fn vertex_main(
  @builtin(vertex_index) vertex_index : u32
) -> FragmentInput {

  var pos = array<vec3f, 3>(
    vec3f( 0.0,  0.5, 0.),
    vec3f( -0.5, -0.5, 0.),
    vec3f( 0.5, -0.5, 0.)
  );

  var color = array<vec4f, 3>(
    vec4f(1., 0., 0., 1.),
    vec4f(0., 1., 0., 1.),
    vec4f(0., 0., 1., 1.)
  );
  
  return FragmentInput( cameraMapPoint(pos[vertex_index]), color[vertex_index] );
}

@fragment
fn fragment_main(input: FragmentInput) -> @location(0) vec4f {
  return input.color;
}
"""


class TriangleCamera(Renderer):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.n_vertices = 3

    def get_bounding_box(self):
        return ((-0.5, -0.5, 0), (0.5, 0.5, 0))

    def get_shader_code(self):
        return shader_code


wj.Draw(TriangleCamera())

## Moving Data to GPU - Bindings

Usually ``Renderer`` objects need data from Python (like vertices, ...) for
this we provide utility functions that can be called in a
``Renderer.update`` method.

GPU objects are connected by using the same binding number within Python and
WGSL.

It is important to not use numbers multiple times; the first 100 are advised
to be left for internal stuff like camera settings, etc.

**Changes in shader code**

Declare global storage buffers at the top:

.. code-block:: rust

   @group(0) @binding(101) var<storage> vertices: array<f32>;
   @group(0) @binding(102) var<storage> colors: array<f32>;

Access buffers in the vertex shader:

.. code-block:: rust

   var pos = vec3f(vertices[3*vertex_index],
                   vertices[3*vertex_index+1],
                   vertices[3*vertex_index+2]);
   var color = vec4f(colors[3*vertex_index],
                     colors[3*vertex_index+1],
                     colors[3*vertex_index+2], 1.);


In [ ]:
import webgpu.jupyter as wj
from webgpu.renderer import Renderer
from webgpu.utils import BufferBinding, buffer_from_array
import numpy as np

shader_code = """
#import camera

// Data structure which is the output of the vertex shader and the input of the fragment shader
struct FragmentInput {
    @builtin(position) p: vec4<f32>,
    @location(0) color: vec4<f32>,
};

@group(0) @binding(101) var<storage> vertices: array<f32>;
@group(0) @binding(102) var<storage> colors: array<f32>;

// Vertex shader, returns a FragmentInput object
@vertex
fn vertex_main(
  @builtin(vertex_index) vertex_index : u32
) -> FragmentInput {

  var pos = vec3f(vertices[3*vertex_index],  vertices[3*vertex_index+1], vertices[3*vertex_index+2]);
  var color = vec4f(colors[3*vertex_index],  colors[3*vertex_index+1], colors[3*vertex_index+2], 1.);
  
  return FragmentInput( cameraMapPoint(pos), color );
}

@fragment
fn fragment_main(input: FragmentInput) -> @location(0) vec4f {
  return input.color;
}
"""


class TriangleInput(Renderer):
    def __init__(self, points, colors, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.n_vertices = 3

        # store GPU data as float32 arrays
        self.points = np.array(points, dtype=np.float32).reshape(-1, 3)
        self.colors = np.array(colors, dtype=np.float32).reshape(-1, 3)

    def update(self, options):
        # copy data to the GPU
        self.gpu_points = buffer_from_array(self.points)
        self.gpu_colors = buffer_from_array(self.colors)

    def get_bounding_box(self):
        return np.min(self.points, axis=0), np.max(self.points, axis=0)

    def get_shader_code(self):
        return shader_code

    def get_bindings(self):
        # associate gpu buffer data with binding numbers
        return [BufferBinding(101, self.gpu_points), BufferBinding(102, self.gpu_colors)]


points = [[0, 0.5, 0], [-0.5, -0.5, 0], [0.5, -0.5, 0]]
colors = [[1, 0, 0], [0, 1, 0], [0, 0, 1]]

wj.Draw(TriangleInput(points, colors))

## Controls

Adding options to the control sections of a scene can be done by the `scene.gui` object.

In [ ]:
triangle = TriangleInput(points, colors)
scene = wj.Draw(triangle)


def move_point(xval):
    triangle.points[0][0] = xval
    scene.redraw()


if scene.gui:
    scene.gui.slider(value=0.0, func=move_point, min=-0.5, max=0.5, label="Position");